In [270]:
%reset -f

In [271]:
from IPython.display import display, HTML
from sklearn import metrics
from sklearn import cross_validation
from scipy import interp
import pandas as pd
import numpy as np
import os

import sys
sys.path.insert(1,"../../src/")
from TypeFeatImputer import TypeFeatImputer
from i_score_parallel import i_score
from TypeFeatFS import DiscreteFS, ContinuousFS
from TypeFeatFilter import DiscreteFilter, ContinuousFilter
from OutlierFiltering import OutlierFiltering
from typeFeat_score import typeFeat_score

from pylab import *
import seaborn as sns
import matplotlib.pyplot as plt


np.set_printoptions(suppress=True)
pd.options.display.float_format = '{:,.4f}'.format
plt.style.use('classic')

%matplotlib inline

In [302]:
ddf = []
pathDir = os.path.join('resources','results') 
path, dirs, files = os.walk(pathDir).next()
file_count = len(files)

for f in files:
    if "summary" not in str(f):
        ddf.append(pd.read_pickle(os.path.join('resources','results', str(f) )))
    
dfAll = pd.concat(ddf)
print dfAll.shape

(8, 35)


In [303]:
dfAll.cv_f1_mean = pd.to_numeric(dfAll.cv_f1_mean)
dfAll.cv_auc_mean = pd.to_numeric(dfAll.cv_auc_mean)
dfAll.cv_sens_mean = pd.to_numeric(dfAll.cv_sens_mean)
dfAll.cv_spec_mean = pd.to_numeric(dfAll.cv_spec_mean)

dfAll.cv_f1_std = pd.to_numeric(dfAll.cv_f1_std)
dfAll.cv_auc_std = pd.to_numeric(dfAll.cv_auc_std)
dfAll.cv_sens_std = pd.to_numeric(dfAll.cv_sens_std)
dfAll.cv_spec_std = pd.to_numeric(dfAll.cv_spec_std)
dfAll.exp = pd.to_numeric(dfAll.exp)

dfAll["cv_auc"] = dfAll["cv_auc_mean"].round(3).astype(str).str.cat(dfAll["cv_auc_std"].round(3).astype(str), sep="+/-")
dfAll["cv_f1"] = dfAll["cv_f1_mean"].round(3).astype(str).str.cat(dfAll["cv_f1_std"].round(3).astype(str), sep="+/-")
dfAll["cv_sens"] = dfAll["cv_sens_mean"].round(3).astype(str).str.cat(dfAll["cv_sens_std"].round(3).astype(str), sep="+/-")
dfAll["cv_spec"] = dfAll["cv_spec_mean"].round(3).astype(str).str.cat(dfAll["cv_spec_std"].round(3).astype(str), sep="+/-")
dfAll["disease"] = dfAll["name"].str.slice(0,dfAll["name"].str.find("_").values[0])

print dfAll.columns.tolist()
print dfAll.name.unique()
print dfAll["disease"].unique()
print dfAll.shape

['exp', 'name', 'size_tr', 'fs', 'sm', 'cls', 'metric', 'params', 'tr_sens', 'tr_spec', 'tr_auc', 'tr_prec', 'tr_rec', 'tr_f1', 'cv_sens_mean', 'cv_sens_std', 'cv_spec_mean', 'cv_spec_std', 'cv_auc_mean', 'cv_auc_std', 'cv_prec_mean', 'cv_prec_std', 'cv_rec_mean', 'cv_rec_std', 'cv_f1_mean', 'cv_f1_std', 'test_sens', 'test_spec', 'test_auc', 'test_rec', 'test_prec', 'test_f1', 'cm_test', 'time', 'pipeline', 'cv_auc', 'cv_f1', 'cv_sens', 'cv_spec', 'disease']
['subset_non_extended_disease_last_early_readmission_vs_none_none']
['subset']
(8, 40)


In [304]:
disease = "subset"
typeHypothesis = "early_readmission_vs_none"
typeDataFeatures = "non_extended"
typeDiagnosis = "none"
typeDataExperiment = "disease"
typeEncounter = "last"

name = disease + "_" + typeDataFeatures + "_" +  typeDataExperiment + "_" + typeEncounter + "_" + \
       typeHypothesis + "_" + typeDiagnosis

print name

subset_non_extended_disease_last_early_readmission_vs_none_none


In [305]:
#Sort and list all pipelines
dfAll.sort_values(["cv_auc_mean"], ascending=False,inplace=True)

dfAll[["metric","cls","params",
    "tr_auc","tr_sens","tr_spec",
    "cv_auc_mean","cv_f1_mean","cv_sens_mean","cv_spec_mean",
    "test_auc","test_f1","test_sens","test_spec"]]

,metric,cls,params,tr_auc,tr_sens,tr_spec,cv_auc_mean,cv_f1_mean,cv_sens_mean,cv_spec_mean,test_auc,test_f1,test_sens,test_spec
0,f1,logReg,"[balanced, 0.05, l1]",0.6272,0.5932,0.6611,0.6650,0.7037,0.5738,0.6589,0.6236,0.7063,0.5868,0.6604
0,f1,rf,"[gini, 8, 500]",0.6689,0.6466,0.6913,0.6623,0.7228,0.5263,0.6952,0.6228,0.7169,0.5662,0.6794
0,recall,rf,"[entropy, 4, 300]",0.6288,0.6122,0.6454,0.6612,0.6965,0.5871,0.6462,0.6240,0.6985,0.6012,0.6467
0,f1_weighted,logReg,"[None, 1e-05, l2]",0.5158,0.0367,0.9949,0.6544,0.8157,0.0346,0.9934,0.5155,0.8169,0.0356,0.9953
0,recall,logReg,"[balanced, 1e-05, l2]",0.6185,0.6625,0.5744,0.6539,0.6513,0.6524,0.5737,0.6155,0.6546,0.6529,0.5780
0,f1_weighted,rf,"[entropy, None, 200]",1.0000,1.0000,1.0000,0.6464,0.8102,0.0086,0.9985,0.5035,0.8103,0.0083,0.9987
0,recall,knn,"[uniform, 1]",1.0000,1.0000,1.0000,0.5296,0.7895,0.1732,0.8861,0.5341,0.7940,0.1746,0.8936
0,f1,knn,"[uniform, 1]",1.0000,1.0000,1.0000,0.5296,0.7895,0.1732,0.8861,0.5341,0.7940,0.1746,0.8936


In [306]:
#Filter pipelines less performance than random
dfAll[np.logical_and(np.logical_and(dfAll['name'] == name, dfAll["cv_sens_mean"] > 0.5),
                     dfAll['cv_spec_mean']> 0.5)][[
    "metric","cls","params",
    "tr_auc","tr_sens","tr_spec",
    "cv_auc_mean","cv_f1_mean","cv_sens_mean","cv_spec_mean",
    "test_auc","test_f1","test_sens","test_spec"]]

,metric,cls,params,tr_auc,tr_sens,tr_spec,cv_auc_mean,cv_f1_mean,cv_sens_mean,cv_spec_mean,test_auc,test_f1,test_sens,test_spec
0,f1,logReg,"[balanced, 0.05, l1]",0.6272,0.5932,0.6611,0.6650,0.7037,0.5738,0.6589,0.6236,0.7063,0.5868,0.6604
0,f1,rf,"[gini, 8, 500]",0.6689,0.6466,0.6913,0.6623,0.7228,0.5263,0.6952,0.6228,0.7169,0.5662,0.6794
0,recall,rf,"[entropy, 4, 300]",0.6288,0.6122,0.6454,0.6612,0.6965,0.5871,0.6462,0.6240,0.6985,0.6012,0.6467
0,recall,logReg,"[balanced, 1e-05, l2]",0.6185,0.6625,0.5744,0.6539,0.6513,0.6524,0.5737,0.6155,0.6546,0.6529,0.5780


#### Grouped by Diseases

In [269]:
dfAll.sort_values(["typeDisease","cv_f1_mean"], ascending=False,inplace=True)
dfAll.groupby(["typeDisease"])[["size_tr","metric","cls","params",
     'tr_f1', 'tr_prec', 'tr_rec', 
     'cv_f1_mean', 'cv_prec_mean', 'cv_rec_mean', 
     'test_f1', 'test_prec', 'test_rec', 'test_auc']].first()

,size_tr,metric,cls,params,tr_f1,tr_prec,tr_rec,cv_f1_mean,cv_prec_mean,cv_rec_mean,test_f1,test_prec,test_rec,test_auc
typeDisease,,,,,,,,,,,,,,
Circulatory_1,1.0000,f1_weighted,logReg,"[None, 0.0001, l2]",0.7974,0.7859,0.8493,0.7968,0.7809,0.8455,0.7992,0.8501,0.7907,0.5199
Diabetis_1,1.0000,f1_weighted,logReg,"[None, 0.0005, l2]",0.8362,0.8311,0.8626,0.8316,0.8226,0.8516,0.8341,0.8592,0.8271,0.5841
Digestive_1,1.0000,f1_weighted,logReg,"[None, 0.0001, l2]",0.8474,0.8383,0.8766,0.8369,0.8220,0.8661,0.8394,0.8669,0.8246,0.5411
Genitourinary_1,1.0000,f1_weighted,logReg,"[None, 0.0005, l2]",0.8293,0.8243,0.8701,0.8237,0.8113,0.8622,0.8293,0.8699,0.8221,0.5312
Muscoskeletal_1,1.0000,f1_weighted,logReg,"[None, 0.001, l2]",0.8410,0.8665,0.8878,0.8344,0.8199,0.8816,0.8314,0.8799,0.8013,0.5012
Neoplasms_1,1.0000,f1_weighted,logReg,"[None, 0.5, l2]",0.8480,0.8749,0.8935,0.8450,0.8232,0.8890,0.8444,0.8889,0.8392,0.5127
Poisoning_1,1.0000,f1_weighted,logReg,"[None, 1e-05, l2]",0.8003,0.7859,0.8310,0.7926,0.7760,0.8239,0.8017,0.8314,0.7877,0.5515
Respiratory_1,1.0000,f1_weighted,logReg,"[None, 0.0001, l2]",0.8312,0.8198,0.8552,0.8186,0.8050,0.8396,0.8148,0.8371,0.7995,0.5418
subset,1.0000,f1_weighted,logReg,"[None, 1e-05, l2]",0.8214,0.8130,0.8639,0.8206,0.8087,0.8611,0.8223,0.8641,0.8142,0.5306


In [161]:
dfAll[dfAll["typeDisease"] == "Neoplasms_1"][["size_tr","sm","metric","cls","params",
     'tr_f1', 'tr_prec', 'tr_rec', 
     'cv_f1_mean', 'cv_prec_mean', 'cv_rec_mean', 
     'test_f1', 'test_prec', 'test_rec', 'test_auc']]

,size_tr,sm,metric,cls,params,tr_f1,tr_prec,tr_rec,cv_f1_mean,cv_prec_mean,cv_rec_mean,test_f1,test_prec,test_rec,test_auc
0,0.40,none,f1_weighted,rf,"[gini, 6, balanced, 100]",0.83,0.83,0.83,0.64,0.63,0.65,0.62,0.63,0.62,0.54
0,0.20,none,f1_weighted,logReg,"[None, 0.0001, l2]",0.66,0.67,0.71,0.64,0.65,0.69,0.63,0.68,0.63,0.54
0,0.60,none,f1_weighted,rf,"[gini, 6, balanced, 250]",0.82,0.82,0.82,0.64,0.63,0.64,0.62,0.62,0.62,0.55
0,0.40,none,recall,rf,"[entropy, 4, balanced, 250]",0.74,0.75,0.74,0.63,0.64,0.62,0.61,0.61,0.62,0.55
0,0.20,none,roc_auc,logReg,"[None, 0.005, l2]",0.64,0.67,0.71,0.63,0.64,0.69,0.63,0.69,0.63,0.53
0,0.40,none,roc_auc,nb,[],0.63,0.63,0.68,0.62,0.61,0.66,0.63,0.67,0.62,0.54
0,0.40,none,recall,nb,[],0.63,0.63,0.68,0.62,0.61,0.66,0.63,0.67,0.62,0.54
0,0.40,none,f1_weighted,nb,[],0.63,0.63,0.68,0.62,0.61,0.66,0.63,0.67,0.62,0.54
0,0.20,none,f1_weighted,rf,"[gini, 6, balanced, 200]",0.88,0.88,0.88,0.62,0.61,0.62,0.61,0.61,0.60,0.53
0,0.60,none,f1_weighted,logReg,"[balanced, 0.05, l2]",0.63,0.67,0.61,0.61,0.65,0.59,0.57,0.55,0.61,0.54


#### Save results

In [211]:
import time
dfAll.iloc[:,:-4].to_pickle(os.path.join(pathDir, "summary" + time.strftime("%Y%m%d-%H%M%S") + ".pkl"))

In [213]:
dd = pd.read_pickle(os.path.join(pathDir,"summary20171017-171147.pkl"))
print dd.columns

Index([u'exp', u'typeDisease', u'typeEncounter', u'typeHypothesis',
       u'typeDataFeatures', u'typeDiagnosis', u'size_tr', u'fs', u'sm', u'cls',
       u'metric', u'params', u'tr_f1', u'tr_prec', u'tr_rec', u'cv_f1_mean',
       u'cv_f1_std', u'cv_prec_mean', u'cv_prec_std', u'cv_rec_mean',
       u'cv_rec_std', u'test_f1', u'test_prec', u'test_rec', u'test_auc',
       u'time', u'pipeline'],
      dtype='object')
